In [1]:
from graphs import execute_graph

Python REPL can execute arbitrary code. Use with caution.


In [2]:
import text
from langchain_core.messages import HumanMessage, AIMessage
function_detail=text.function_detail
task=text.task
plan=text.plan
session_id = '1'

In [3]:
# initiate empty list to collect successfully executed code
successful_code = []

inputs = {"plan": plan, "task": task, "function_detail":function_detail, "session_id": session_id, 'messages':[], 'successful_code': []}

for s in execute_graph.graph.stream(inputs, {"recursion_limit": 100}):
    if "__end__"  in s:
        final_result = s['__end__']['final_result']
        successful_code = s['__end__']['successful_code']

Executing: 
# Import necessary libraries
from pybaseball import statcast

# Get all pitch data from 2020-08-01 to 2020-08-07
all_pitches = statcast('2020-08-01', '2020-08-07')



100%|██████████| 7/7 [00:04<00:00,  1.48it/s]


Result: This is a large query, it may take a moment to complete

Executing: 
# Filter for just curveballs
all_curves = all_pitches[all_pitches['pitch_type'] == 'CU']

Executing: 
# Create a feature vector for each pitcher's curveballs
pitcher_curves = all_curves.groupby('pitcher')
pitcher_features = pitcher_curves[['release_speed', 'release_spin', 'pfx_x', 'pfx_z']].mean().reset_index()

Result: KeyError("Columns not found: 'release_spin'")
Executing: 
print(all_curves.columns)

Result: Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up'

In [4]:
print(final_result)


The 3 pitchers in the first week of August 2020 whose curveballs were most similar to Max Scherzer's based on release speed, spin rate, horizontal and vertical movement were:

Pitcher ID 593833 (likely Zach Plesac of the Cleveland Indians)
Pitcher ID 664285 (likely Kris Bubic of the Kansas City Royals) 
Pitcher ID 608331 (likely Zach Davies of the San Diego Padres)



In [6]:
print(''.join(successful_code))


# Import necessary libraries
from pybaseball import statcast

# Get all pitch data from 2020-08-01 to 2020-08-07
all_pitches = statcast('2020-08-01', '2020-08-07')

# Filter for just curveballs
all_curves = all_pitches[all_pitches['pitch_type'] == 'CU']

print(all_curves.columns)

# Create a feature vector for each pitcher's curveballs 
pitcher_curves = all_curves.groupby('pitcher')
pitcher_features = pitcher_curves[['release_speed', 'release_spin_rate', 'pfx_x', 'pfx_z']].mean().reset_index()

from pybaseball import playerid_lookup

# Get Max Scherzer's player ID
scherzer_id = playerid_lookup('scherzer', 'max')['key_mlbam'][0]


# Get Scherzer's feature vector
scherzer_features = pitcher_features[pitcher_features['pitcher'] == scherzer_id]


from sklearn.neighbors import NearestNeighbors

# Import scikit-learn and create a knn model
knn = NearestNeighbors(n_neighbors=4)
knn.fit(pitcher_features[['release_speed', 'release_spin_rate', 'pfx_x', 'pfx_z']])


# Find the 3 pitchers closest